In [1]:
import pandas as pd
import numpy as np
import psycopg2
from sentence_transformers import SentenceTransformer

/home/user03/Documents/SVN/UrFU.term3/Hackaton2023/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
vk_faculty = 'Факультет биоинженерии и биоинформатики'

select_query = 'select * from (select embedding <-> %s as cos, d.direction_id, d.value from h_edu_direction d ORDER BY d.embedding <-> %s limit 2) d'

coords = (53.507852, 49.420411)

In [3]:
select_query = ('select * from ' + 
                ' (select cos, faculty, full_name, city, dist, ' +
                '     row_number() over (partition by faculty order by dist) univ_dist_rank ' +
                '     from ' +
                '   (select distinct d.cos, d.faculty, u.full_name, u.city, ' +
                '     get_haversine_distance(u.latitude, u.longitude, %s, %s) dist ' +
                '    from ' +
                '     (select d.direction_id, d.value as faculty, '
                '       embedding <-> %s as cos '
                '      from h_edu_direction d order by d.embedding <-> %s limit 2) d, ' +
                '     h_edu_direct2univ d2u, h_edu_universities u ' +
                '    where d2u.direction_id = d.direction_id and u.university_id = d2u.university_id) u) u ' +
                'where univ_dist_rank < 10 ' +
                'order by faculty, univ_dist_rank, full_name')

In [4]:
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

/home/user03/Documents/SVN/UrFU.term3/Hackaton2023/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [5]:
embeddings4select = embedding_model.encode(vk_faculty)
embeddings4select = '[' + (','.join(map(str, embeddings4select))) + ']'

In [6]:
conn = psycopg2.connect(dbname="h_core", user="h_user", password="AUdZuMOGuP", host="84.201.178.7", port="55432")

In [7]:
with conn.cursor() as curs:
    curs.execute(select_query,(coords[0], coords[1], embeddings4select,embeddings4select))
    rs = curs.fetchall()

In [8]:
rs

[(2.5999883064593936,
  'Биологические науки',
  'федеральное государственное бюджетное образовательное учреждение высшего образования «Самарский государственный медицинский университет» Министерства здравоохранения Российской Федерации',
  'Самара',
  57.326245444075575,
  1),
 (2.5999883064593936,
  'Биологические науки',
  'федеральное государственное бюджетное образовательное учреждение высшего образования «Самарский государственный социально-педагогический университет»',
  'Самара',
  57.326245444075575,
  2),
 (2.5999883064593936,
  'Биологические науки',
  'федеральное государственное автономное образовательное учреждение высшего образования «Самарский национальный исследовательский университет имени академика С.П. Королева»',
  'Самара',
  57.326245444075575,
  3),
 (2.5999883064593936,
  'Биологические науки',
  'федеральное государственное бюджетное образовательное учреждение высшего образования «Самарский государственный аграрный университет»',
  'Кинель',
  86.6325704108776

In [9]:
conn.close()